## TP 2: Transformers

Pour ce TP, nous continuons à travailler sur une tâche de classification multi-classes. Cepandant, cette fois-ci, nous allons utiliser les modèles Transformers.

<font color='red'>NB: Il est forcement recommandé de lancer ce notebook dans Colab ou sur votre machine à cause des ressources de GPU requierts pour entraîner les modèles aussi que les problèmes potentiels avec l'installation des bibliotèques. Si vous voulez utilisez votre propre machine pour ce TP et vous n'avez pas de carte vidéo, le modèle déjà entraîné est fourni plus loin dans le notebook.

Pour ouvrir ce notebook dans Colab, suivez ce lien: https://colab.research.google.com/drive/1PgAtwKy43IWQVRqhPvZO4ZZHrefyvvAH?usp=sharing.</font>

In [ ]:
from datasets import load_dataset

dataset = load_dataset("tweet_eval", 'emotion')

In [ ]:
dataset['train'][0]

In [ ]:
from transformers import AutoTokenizer

model_name = "distilbert-base-cased"

tokenizer = AutoTokenizer.from_pretrained(model_name)

In [ ]:
tokenized_sample = tokenizer(dataset['train'][0]['text'])

In [ ]:
tokenized_sample

In [ ]:
tokenizer.convert_ids_to_tokens(tokenized_sample['input_ids'])

### Question 1

Donnez deux textes à la fois (c.-à-d. à deux paramètres pour `tokenizer`) au tokenizer et regardez la sortie. Qu'est-ce qu'il est différent de la sortie précédente ? Pensez à une tâche qui peut requérir ce genre de tokenisation.

In [ ]:
tokenized_sample_2 = tokenizer(dataset['train'][1]['text'], dataset['train'][2]['text'])
tokenizer.convert_ids_to_tokens(tokenized_sample_2['input_ids'])

In [ ]:
def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True)

tokenized_datasets = dataset.map(tokenize_function, batched=True)

In [ ]:
print(tokenized_datasets['train'][0])

In [ ]:
num_labels = dataset['train'].info.features['label'].num_classes

Initialisons le modèle pre-entraîné.

In [ ]:
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=num_labels)

In [ ]:
from transformers import TrainingArguments

batch_size = 16

training_args = TrainingArguments(
    f"{model_name}-finetuned-tweeteval",
    evaluation_strategy = "epoch",
    save_strategy = "epoch",
    learning_rate=2e-5,
    optim='adamw_torch',
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=5,
    weight_decay=0.01,
    load_best_model_at_end=True,
    metric_for_best_model="accuracy",
    greater_is_better=True,
)

### Question 2

Lisez la documentation de [TrainingArguments](https://huggingface.co/docs/transformers/v4.28.1/en/main_classes/trainer#transformers.TrainingArguments). Décrivez tous les paramètres d'entraînement ci-dessus. Selon la documentation, quels sont les autres paramètres qui peuvent influencer l'entraînement de modèle le plus ?

Après, n'hésitez pas à changer ou ajouter les paramètres selon vos préferences. 

Pour calculer les métriques, nous utilisons la bibliotèque [evaluate](https://huggingface.co/docs/evaluate/index). Pour l'accuracy, sklearn est utilisé "sous le capot".

In [ ]:
import numpy as np
import evaluate

metric = evaluate.load("accuracy")

In [ ]:
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

_NB: Si vous n'avez pas de ressources pour l'entraînement ou vous ne voulez pas attendre, vous pouvez sauter cette étape de l'entraînement et télécharger le modèle déjà entraîner dans le code qui suit._

In [ ]:
from transformers import Trainer

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets['train'],
    eval_dataset=tokenized_datasets['validation'],
    compute_metrics=compute_metrics,
)

Une bonne chose à faire, c'est de créer aussi la carte de modèle et sauvgarder le tokenizer. Comme ça, vous pouver partager très facilement votre modèle et l'utiliser après.

Regardez les autres paramètres de [`create_model_card`](https://huggingface.co/docs/transformers/v4.28.1/en/main_classes/trainer#transformers.Trainer.create_model_card) et n'hésitez pas à les changer selon vos préferences.

In [ ]:
trainer.train()

trainer.save_model()
trainer.create_model_card(language='en', dataset='tweet_eval')
tokenizer.save_pretrained(f"{model_name}-finetuned-tweeteval")

Décommentez la ligne ci-dessous pour télécharger le modèle déjà entraîné sur le même jeu de données.

In [ ]:
model = AutoModelForSequenceClassification.from_pretrained("501Good/distilbert-base-cased-finetuned-tweeteval")

## Question 3

Comme dans le TP1, on peut aussi visualiser l'attention.

Faites la prédiction du premier texte de "test set" avec le modèle entraîné en sauvgardant toujours les attentions de modèle (consultez la documentation de [`DistilBertForSequenceClassification`](https://huggingface.co/docs/transformers/v4.28.1/en/model_doc/distilbert#transformers.DistilBertForSequenceClassification)). Après, visualisez l'attention avec `head_view` et `model_view` dans le Bertviz. 

In [ ]:
from bertviz import head_view, model_view
import torch

In [ ]:
text = dataset['test'][0]['text']
label = dataset['test'][0]['label']

inputs = tokenizer(text, return_tensors='pt')
outputs = model(**inputs, output_attentions=True)

tokens = tokenizer.convert_ids_to_tokens(inputs['input_ids'][0])
head_view(outputs.attentions, tokens)

In [ ]:
print("La phrase est : ", text)
print("L'emotion de la phrase est : ", label)
print("La prédiction du modèle est : ", model(**inputs).logits.argmax().item())

In [ ]:
acc = 0
for i in range(100):
    text = dataset['test'][i]['text']
    label = dataset['test'][i]['label']

    inputs = tokenizer(text, return_tensors='pt')
    outputs = model(**inputs, output_attentions=True)

    tokens = tokenizer.convert_ids_to_tokens(inputs['input_ids'][0])

    if label == model(**inputs).logits.argmax().item():
        acc += 1
print("L'accuracy est de : ", acc/100)

In [ ]:
model_view(outputs.attentions, tokens)

## Exercice

L'intégration de connaissances externes dans les modèles est un moyen possible d'améliorer les performances des modèles. Une possibilité consiste à marquer le texte annoté dans le texte d'entrée, ce qui permet au modèle d'accorder plus d'attention à ces passages annotés.

Utiliser le lexique des émotions fourni et marquer les mots correspondants dans le texte d'entrée. Faire des prédictions basées sur le texte modifié et analyser les résultats.

A la fin du cours (pendant le dernier TP), vous devez présenter vos résultats et votre analyse pour les exercices (TP2 et TP4). Il s'agira d'une courte présentation de 5 à 10 minutes et vous devrez soumettre les diapositives correspondantes pour évaluation avec le code. Les deux exercices seront notés sur 10 points et le score total sera ajouté à la note finale du cours.

In [ ]:
import pandas as pd
lexique_emotion = pd.read_csv('NRC-Emotion-Lexicon-Wordlevel-v0.92.txt', sep='\t', header=None, names=['word', 'emotion', 'value'])
lexique_emotion.head()


In [ ]:
def annotate_text(texts,lexicon):
    dataset = []
    for text in texts:
        words = text.split()
        annotate_text = []
        
        for word in words:
            if word.lower() in lexicon.word.values:
                annotate_text.append("[SENT] " + word + " [SENT]")
            else:
                annotate_text.append(word)
        new_text = ' '.join(annotate_text)
        dataset.append(new_text)
    return dataset

new_dataset_train = dataset['train'].map(lambda x: {'text': annotate_text(x['text'], lexique_emotion)}, batched=True)
            


In [ ]:
new_dataset_eval = dataset['validation'].map(lambda x: {'text': annotate_text(x['text'], lexique_emotion)}, batched=True)

In [ ]:
new_tokenized_datasets_train = new_dataset_train.map(tokenize_function, batched=True)
new_tokenized_datasets_eval = new_dataset_eval.map(tokenize_function, batched=True)

In [ ]:
new_tokenized_datasets_train.set_format(type='torch', device='cuda')
new_tokenized_datasets_eval.set_format(type='torch', device='cuda')

In [ ]:
from transformers import TrainingArguments

batch_size = 16

training_args = TrainingArguments(
    f"{model_name}-finetuned-tweeteval",
    evaluation_strategy = "epoch",
    save_strategy = "epoch",
    learning_rate=2e-5,
    optim='adamw_torch',
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=5,
    weight_decay=0.01,
    load_best_model_at_end=True,
    metric_for_best_model="accuracy",
    greater_is_better=True,
)

In [ ]:
from transformers import Trainer 

if(torch.cuda.is_available()):
    device = torch.device("cuda")
    print("There are %d GPU(s) available." % torch.cuda.device_count())
else : 
    print("No GPU available, using the CPU instead.")
    device = torch.device("cpu")

model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=num_labels)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=new_tokenized_datasets_train,
    eval_dataset=new_tokenized_datasets_eval,
    compute_metrics=compute_metrics,
)
trainer.train()

trainer.save_model()
trainer.create_model_card(language='en', dataset='tweet_eval')
tokenizer.save_pretrained(f"{model_name}-finetuned-tweeteval")

In [ ]:
acc = 0
for i in range(100):
    text = annotate_text(dataset['test'][i]['text'], lexique_emotion)
    label = dataset['test'][i]['label']

    inputs = tokenizer(text, return_tensors='pt')
    outputs = model(**inputs, output_attentions=True)

    tokens = tokenizer.convert_ids_to_tokens(inputs['input_ids'][0])

    if label == model(**inputs).logits.argmax().item():
        acc += 1
print("L'accuracy est de : ", acc/100)